In [2]:
import json
from datetime import datetime
from collections import defaultdict

data_path = "cleaned_esport_data.json"

with open(data_path, 'r') as file:
        esports_data = json.load(file)

def calculate_shifted_team_ratings(data_path, date_str):
    # Load the data from the JSON file
    #with open(data_path, 'r') as file:
        #esports_data = json.load(file)

    # Convert the input date string to a datetime object
    input_date = datetime.strptime(date_str, '%Y-%m-%d')

    # Sort the games by eventTime in descending order
    sorted_games = sorted(esports_data, key=lambda x: x['eventTime'], reverse=True)

    # Initialize a dictionary to store the gold differences for each team
    team_gold_diffs = defaultdict(list)

    # Iterate through the sorted games to calculate gold differences
    for game in sorted_games:
        event_time = datetime.fromisoformat(game['eventTime'].replace("Z", ""))

        # Only consider games before or on the given date
        if event_time <= input_date:
            try:
                team_mappings = game['teamMappings']
            except KeyError:
                continue  # Skip this game if 'teamMappings' key is not present

            # Ensure there are two teams in the game
            if len(team_mappings) == 2:
                team_1, team_2 = team_mappings
                gold_diff = team_1['totalGold'] - team_2['totalGold']

                # Assign gold differences based on winning and losing
                if gold_diff > 0:
                    team_gold_diffs[team_1['name']].append(gold_diff)
                    team_gold_diffs[team_2['name']].append(-gold_diff)
                elif gold_diff < 0:
                    team_gold_diffs[team_2['name']].append(-gold_diff)
                    team_gold_diffs[team_1['name']].append(gold_diff)

    # Compute the ratings by summing the gold differences of the last 30 games
    team_ratings = {}
    for team, gold_diffs in team_gold_diffs.items():
        if len(gold_diffs) >= 15:
            team_ratings[team] = sum(gold_diffs[:30])
    
    # Diagnostic print statements
    #print(f"Number of teams processed: {len(team_gold_diffs)}")
    #print(f"Number of teams with 15 or more games: {len(team_ratings)}")
    
    if not team_ratings:  # Check if team_ratings is empty
        return {}

    # Shift all ratings by the absolute value of the minimum rating
    min_rating = min(team_ratings.values())
    shifted_team_ratings = {team: rating + abs(min_rating) for team, rating in team_ratings.items()}

    # Sort the teams by shifted ratings in descending order
    sorted_shifted_team_ratings = {k: v for k, v in
                                   sorted(shifted_team_ratings.items(), key=lambda item: item[1], reverse=True)}

    return sorted_shifted_team_ratings


# You can call this function with the path to your data file and a date string in the format 'YYYY-MM-DD'
# Example: calculate_shifted_team_ratings('path_to_your_data_file', '2023-09-01')


import json
from datetime import datetime
from collections import defaultdict


def calculate_region_multiplier(data_path, date_str):
    #with open(data_path, 'r') as file:
        #esports_data = json.load(file)

    input_date = datetime.strptime(date_str, '%Y-%m-%d')
    region_wins = defaultdict(int)
    region_total_games = defaultdict(int)

    for game in esports_data:
        event_time = datetime.fromisoformat(game['eventTime'].replace("Z", ""))
        if event_time <= input_date and game['region'] == "INTERNATIONAL":
            team_mappings = game['teamMappings']
            if len(team_mappings) == 2:
                team_1, team_2 = team_mappings
                region_1, region_2 = team_1['actual_region'], team_2['actual_region']
                gold_diff = team_1['totalGold'] - team_2['totalGold']
                if gold_diff > 0:
                    region_wins[region_1] += 1
                    region_total_games[region_1] += 1
                    region_total_games[region_2] += 1
                elif gold_diff < 0:
                    region_wins[region_2] += 1
                    region_total_games[region_1] += 1
                    region_total_games[region_2] += 1

    region_multipliers = {}
    for region in region_total_games.keys():  # Iterate over all regions including None
        wins = region_wins.get(region, 0)
        total_games = region_total_games.get(region, 0)
        if region is None:  # Check if region is None
            region_multipliers[region] = 0.3
        else:
            winrate = wins / total_games if total_games > 0 else 0
            region_multipliers[region] = (1 + (winrate - 0.5))**2

    return region_multipliers


import json
from datetime import datetime
from collections import defaultdict

def calculate_team_multiplier(data_path, date_str):
    #with open(data_path, 'r') as file:
        #esports_data = json.load(file)

    input_date = datetime.strptime(date_str, '%Y-%m-%d')
    team_wins = defaultdict(int)
    team_total_games = defaultdict(int)
    all_teams = set()  # Keep track of all unique teams
    
    for game in esports_data:
        event_time = datetime.fromisoformat(game['eventTime'].replace("Z", ""))
        if event_time <= input_date:
            team_mappings = game['teamMappings']
            if len(team_mappings) == 2:
                team_1, team_2 = team_mappings
                team_name_1, team_name_2 = team_1['name'], team_2['name']
                all_teams.add(team_name_1)  # Add teams to the set of all teams
                all_teams.add(team_name_2)
                
                if game['region'] == "INTERNATIONAL":
                    gold_diff = team_1['totalGold'] - team_2['totalGold']
                    if gold_diff > 0:
                        team_wins[team_name_1] += 1
                        team_total_games[team_name_1] += 1
                        team_total_games[team_name_2] += 1
                    elif gold_diff < 0:
                        team_wins[team_name_2] += 1
                        team_total_games[team_name_1] += 1
                        team_total_games[team_name_2] += 1

    team_multipliers = {}
    for team in all_teams:  # Iterate over all unique teams
        wins = team_wins.get(team, 0)
        total_games = team_total_games.get(team, 0)
        winrate = wins / total_games if total_games > 0 else 0
        team_multipliers[team] = (1 + (winrate - 0.5))**2 if total_games > 0 else 0.3  # Assign 0.3 if no international games

    return team_multipliers


# Example usage:
# region_multipliers = calculate_region_multiplier('path_to_your_data_file', '2023-09-01')
# team_multipliers = calculate_team_multiplier('path_to_your_data_file', '2023-09-01')



In [3]:
# region multiplier
region_multiplier = calculate_region_multiplier('cleaned_esport_data.json', '2023-09-01')
print("region multiplier:", region_multiplier)

region multiplier: {'KOREA': 1.354820600239309, 'HONG KONG, MACAU, TAIWAN': 0.8536153119092628, 'CHINA': 1.3366284428752206, 'BRAZIL': 0.6944444444444443, 'NORTH AMERICA': 0.8732580261593341, 'EMEA': 0.8754288795216154, 'VIETNAM': 0.5625, 'OCEANIA': 0.573265306122449, 'LATIN AMERICA': 0.49826989619377154, 'COMMONWEALTH OF INDEPENDENT STATES': 0.6747448979591836, None: 0.3, 'JAPAN': 0.7561436672967864}


In [4]:
# team multiplier
team_multiplier = calculate_team_multiplier('cleaned_esport_data.json', '2023-09-01')
print("team multiplier:", team_multiplier)

team multiplier: {'Hanwha Life Esports': 1.1842560553633221, 'BWE Esports': 0.3, 'Team Liquid Honda First': 0.3, 'Machi Esports': 0.4444444444444444, 'CrowCrowd Moscow': 0.3, "Kelyx's Grandpa Gamers": 0.3, 'DREN Esports': 0.3, 'Eclipse Gaming': 0.3, 'BC Academy': 0.3, 'Team E Turner': 0.3, 'Eclot Gaming': 0.3, 'Mousesports': 0.3, 'VCS All Stars': 2.25, 'Kenty': 0.3, 'INTZ Academy': 0.3, 'TEAM WHALES': 0.3, 'Wildcard': 0.3, 'Cloud9': 0.8247605164514785, 'Vivo Keyd Stars': 0.3, 'Team Tony Top': 0.3, 'PEACE': 0.8402777777777779, 'FL Academy': 0.3, 'University of Western Ontario': 0.3, 'MAD Lions Madrid': 0.3, 'Verdant': 0.3, 'Pentagon Rejects': 0.3, 'AXZ Academy': 0.3, 'Split Raiders': 0.3, 'Astralis Talent': 0.3, 'UCAM': 0.3, 'GTZ Esports': 0.3, 'CGA  Academy': 0.3, 'Team Secret': 0.3, 'Orbit Anonymo': 0.3, 'Solary': 0.3, 'Legacy Esports': 0.765625, 'Bay State College': 0.3, 'Beşiktaş': 0.3, 'Defusekids': 0.3, 'WLGaming Esports': 0.3, 'K1CK Neosurf': 0.3, 'Deep Cross Gaming': 0.3, 'Info 

In [5]:

# Global Rankings Example without mutlipliers
global_rankings = calculate_shifted_team_ratings('cleaned_esport_data.json', '2023-09-01')
print("Global Rankings:", global_rankings)




Global Rankings: {'DenizBank İstanbul Wildcats': 689778, 'Macko Esports': 658912, 'Gen.G': 617169, 'SK Gaming Prime': 605030, 'Primate': 598965, 'Entropiq': 585963, 'SHG Academy': 580523, 'PSG Talon': 575895, 'Cloud9 Challengers': 573365, 'Zero Tenacity': 572478, 'GAM Esports': 565941, 'G2 Esports': 562520, 'Fuego': 559690, 'Beijing JDG Intel Esports Club': 559091, 'Bandits Gaming': 557658, 'X7': 551718, 'Royal Youth': 545351, 'Movistar R7': 542896, 'SG Academy': 539629, 'DetonatioN FocusMe': 535041, 'Karmine Corp': 533605, 'Riddle Esports': 532539, 'SuppUp eSports': 530456, 'kt Rolster': 530309, 'T1 Rookies': 522729, 'Mirage Alliance': 519976, 'Cloud9': 519588, 'Team GO': 516182, 'AGO Rogue': 515757, 'Dusty': 511729, 'Disguised': 509223, 'Beyond Gaming': 508043, 'paiN Gaming': 504464, 'Golden Guardians': 503258, 'One Breath Gaming': 502155, 'Machi Esports': 500937, 'Bilibili Gaming Pingan Bank': 498893, 'Diamant Esports': 497806, 'Sengoku Gaming': 497297, 'Team 7AM': 496663, 'CLG Chal

In [6]:
def calculate_adjusted_team_ratings(data_path, date_str):
    
    region_multiplier = calculate_region_multiplier(data_path, date_str)
    team_multiplier = calculate_team_multiplier(data_path, date_str)
    global_shifted_rankings = calculate_shifted_team_ratings(data_path, date_str)
    
      # Diagnostic print statements
   # print(f"Number of teams in shifted_team_ratings: {len(global_adjusted_rankings)}")
   # print(f"Region multipliers: {region_multiplier}")
    #print(f"Team multipliers: {team_multiplier}")

    adjusted_team_ratings = {}
    
    # Load the esports data to get the regions for the teams
    with open(data_path, 'r') as file:
        esports_data = json.load(file)
    
    # Create a dictionary mapping teams to their regions
    team_to_region = {}
    for game in esports_data:
        for team_mapping in game['teamMappings']:
            team_name = team_mapping['name']
            if team_mapping.get('region') == "INTERNATIONAL":
                team_to_region[team_name] = team_mapping.get('actual_region', None)
            else:
                team_to_region[team_name] = team_mapping.get('region', None)
    
    # Adjust the ratings by applying the multipliers
    for team, rating in global_shifted_rankings.items():
        region = team_to_region.get(team, None)
        region_multipliers = region_multiplier.get(region, 1)
        team_multipliers = team_multiplier.get(team, 1)
        
        adjusted_rating = rating * region_multipliers * team_multipliers
        adjusted_team_ratings[team] = adjusted_rating
    
    # Sort the adjusted ratings in descending order
    sorted_adjusted_team_ratings = {k: v for k, v in
                                    sorted(adjusted_team_ratings.items(), key=lambda item: item[1], reverse=True)}
    
    return sorted_adjusted_team_ratings


In [7]:
def calculate_tournament_rankings(data_path, tournament_id):
    with open(data_path, 'r') as file:
        esports_data = json.load(file)

    # Get the last day of the tournament
    tournament_games = [game for game in esports_data if game['tournamentId'] == tournament_id]
    last_game_time = max(game['eventTime'] for game in tournament_games)
    last_day = datetime.fromisoformat(last_game_time.replace("Z", ""))

    # Calculate the adjusted ratings as of the last day of the tournament
    adjusted_ratings = calculate_adjusted_team_ratings(data_path, last_day.strftime('%Y-%m-%d'))

    # Get the names of the teams that participated in the tournament
    tournament_teams = set()
    for game in tournament_games:
        for team_mapping in game['teamMappings']:
            tournament_teams.add(team_mapping['name'])

    # Filter the adjusted ratings to include only the teams that participated in the tournament
    tournament_rankings = {team: rating for team, rating in adjusted_ratings.items() if team in tournament_teams}

    # Sort the teams by adjusted ratings in descending order
    sorted_tournament_rankings = {k: v for k, v in
                                  sorted(tournament_rankings.items(), key=lambda item: item[1], reverse=True)}

    return sorted_tournament_rankings

# Example usage:
# tournament_rankings = calculate_tournament_rankings('path_to_your_data_file', 'your_tournament_id')
# print(tournament_rankings)

In [8]:
def get_team_rankings(data_path, date_str, team_names):
    """
    Get the rankings of specified teams as of a certain date.
    """
    # Get the global rankings using the existing function
    global_rankings = calculate_adjusted_team_ratings(data_path, date_str)

    # Filter the global rankings to get rankings of the specified teams
    team_rankings = {team: rank for team, rank in global_rankings.items() if team in team_names}

    return team_rankings

In [9]:
# Global Rankings Example
global_adjusted_rankings = calculate_adjusted_team_ratings('cleaned_esport_data.json', '2023-09-01')
print(global_adjusted_rankings)

{'Beijing JDG Intel Esports Club': 1095069.466558687, 'Gen.G': 1045468.0080921622, 'Bilibili Gaming Pingan Bank': 887797.5094322672, 'Dplus Kia': 835929.7091412746, 'Suzhou LNG Esports': 766067.2584497978, 'Royal Never Give Up': 758943.7661500057, 'T1': 721373.8909001211, 'Hanwha Life Esports': 717621.1552006113, 'WeiboGaming FAW AUDI': 715601.0533554589, 'G2 Esports': 546621.6223746446, 'TOP ESPORTS': 528772.009327212, 'DRX': 500352.976187166, 'LGD GAMING': 476134.5435792178, 'PSG Talon': 475862.84385625285, 'Golden Guardians': 465705.6725500824, 'Team Liquid Honda': 418817.155643891, 'SHANGHAI EDWARD GAMING HYCAN': 400739.64324833406, 'Cloud9': 374222.210897149, 'Fnatic': 361106.7617329183, 'Evil Geniuses LG': 355618.2222856912, 'Beyond Gaming': 342655.4341991645, 'DetonatioN FocusMe': 320633.87071562634, 'Galatasaray Espor': 319118.34670007834, 'MAD Lions': 282248.40295375447, 'Papara SuperMassive ': 272882.3921000392, 'LOUD': 263021.9618055555, 'FlyQuest': 257080.1768370987, 'paiN 

In [10]:
team_names = ['Bilibili Gaming Pingan Bank', 'Team Liquid Honda First', 'FunPlus Phoenix']
team_rankings = get_team_rankings('cleaned_esport_data.json','2023-09-01', team_names)
print(team_rankings)

{'Bilibili Gaming Pingan Bank': 887797.5094322672, 'FunPlus Phoenix': 146908.7980587511, 'Team Liquid Honda First': 44176.9883079667}


In [11]:
tournament_rankings = calculate_tournament_rankings('cleaned_esport_data.json', '103462459318635408')
print(tournament_rankings)

{'Fnatic': 86273.09999999999, 'G2 Esports': 78005.4, 'Astralis': 73471.2, 'Misfits Gaming': 45435.299999999996, 'MAD Lions': 43845.0, 'KOI': 43296.299999999996, 'Schalke 04 Esports': 32605.8, 'EXCEL': 29003.1, 'SK Gaming': 14920.199999999999, 'Team Vitality': 3539.7}
